In [1]:
import numpy as np
import math
from operator import xor
from functools import reduce
from itertools import compress
from itertools import islice

In [10]:
def lfsr_generator(poly, state=None):
    '''
    Generator implementing a Linear Feedback Shift Register (LFSR)
    
    Args:
        poly: list of int,
            feedback polinomial expressed as list of power exponents
        state: int, optional
            initial state of the LFSR (all 1 if not provided)
            
    Return:
        b: bool,
            generated output bit
    '''
    
    # Degree of LFSR
    length = max(poly)
    poly = [i in poly for i in range(length+1)]
    
    if state is None:
        state = [True for _ in range(length)]
    else:
        state = [bool((state >> bit) & 1) for bit in range(len(bin(state)) - 3, -1, -1)]
        
    output = state[0]
    feedback = reduce(xor, compress(state[::-1], poly[1:]))
    
    while True:
        state = state[1:] + [feedback] # Shift to right and add the feedback
        output = state[0]
        feedback = reduce(xor, compress(state[::-1], poly[1:]))
        print(''.join(str(int(s)) for s in state)[::-1], "  ", int(output), " ", int(feedback))
        yield output

### Produce polynomial vector

In [6]:
poly_int = [3, 1, 0] # Polynomial expressed as list of int corresponding to degree

length = max(poly_int) # Take max-degree+1 to create list

poly = [i in poly_int for i in range(length+1)]
print(poly)

[True, True, False, True]


### Compute state

In [19]:
state = [bool((7 >> bit) & 1) for bit in range(len(bin(7)) - 3, -1, -1)]
state

[True, True, True]

### Compute feedback

In [8]:
fb = reduce(lambda a,b: a^b, list(x^y for x,y in zip(state[::-1], poly[1:])))
fb

True

In [11]:
reduce(xor, compress(state[::-1], poly[1:]))

False

In [11]:
poly_int = [3, 1, 0] # Feedback polynomial (primitive)
state = 0x7      # Initial state 0b111
iters = 7        # Iterations

lfsr = lfsr_generator(poly_int, state) 

print("state  b  fb")
for b in islice(lfsr, iters):
    # do nothing, just let the generator print its internal state
    pass

state  b  fb
011    1   1
101    1   0
010    0   0
001    1   1
100    0   1
110    0   1
111    1   0


# **LFSR Iterator**

In [9]:
class LFSR(object):
    '''
    LFSR Iterator
    '''
    
    def __init__(self, poly, state=None):
        self._poly = sum([2**i for i in poly]) >> 1 # p0 always one
        self._length = max(poly)
        
        # If no state provided initialize with 111...1
        self._smask = (1<<self._length) - 1
        if state is None:
            state = self._smask
        self.state = state
        
        self._output = self._state >> (self._length-1)
        
        self._feedback = self._parity(self._state & self._poly)
    
    def _parity(self, num):
        ite = 1
        ones = 0
        while ite <= num:
            if ite & num:
                ones = ones + 1
            ite = ite << 1
        return ones % 2
    
    # ==== OUTPUT ====
    @property
    def output(self):
        return self._output
    @output.setter
    def output(self, val):
        raise AttributeError('Denied')
        
    # ==== FEEDBACK ====
    @property
    def feedback(self):
        return self._feedback
    @feedback.setter
    def feedback(self, feedback):
        self._feedback = bool(feedback)
        
    # ==== LENGTH ====
    @property
    def length(self):
        return self._length
    @output.setter
    def length(self, val):
        raise AttributeError('Denied')
        
    # ==== STATE ====
    @property
    def state(self):
        # state is re-reversed before being read
        return int(f'{self._state:0{len(self)}b}'[::-1], 2)
    @state.setter
    def state(self, state):
        # ensure seed is in the range [1, 2**m-1]
        #if (state < 1) or (state > len(self)):
            #state = 1 + state % (2**len(self)-2)
        self._state = int(f'{state:0{len(self)}b}'[::-1], 2)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        '''Execute a LFSR step and returns the output bit (bool)'''
        self._state = ((self._state << 1) | self._feedback) & self._smask
        self._output = self._state >> (self._length-1)
        self._feedback = self._parity(self._state & self._poly)
        return bool(self._output)
    
    def __len__(self):
        return self._length
    
    def run_steps(self, N=1):
        '''
        Run given number of steps and return the output stream
        '''
        return [next(self) for _ in range(n)]
    
    def cycle(self, state=None):
        if state is not None:
            self.state = state
        return self.run_steps(2**len(self) - 1)
    
    def __str__(self):
        return Noneclass LFSR(object):
    '''
    LFSR Iterator
    '''
    
    def __init__(self, poly, state=None):
        self._poly = sum([2**i for i in poly]) >> 1 # p0 always one
        self._length = max(poly)
        
        # If no state provided initialize with 111...1
        self._smask = (1<<self._length) - 1
        if state is None:
            state = self._smask
        self.state = state
        
        self._output = self._state >> (self._length-1)
        
        self._feedback = self._parity(self._state & self._poly)
    
    def _parity(self, num):
        ite = 1
        ones = 0
        while ite <= num:
            if ite & num:
                ones = ones + 1
            ite = ite << 1
        return ones % 2
    
    # ==== OUTPUT ====
    @property
    def output(self):
        return self._output
    @output.setter
    def output(self, val):
        raise AttributeError('Denied')
        
    # ==== FEEDBACK ====
    @property
    def feedback(self):
        return self._feedback
    @feedback.setter
    def feedback(self, feedback):
        self._feedback = bool(feedback)
        
    # ==== LENGTH ====
    @property
    def length(self):
        return self._length
    @output.setter
    def length(self, val):
        raise AttributeError('Denied')
        
    # ==== STATE ====
    @property
    def state(self):
        # state is re-reversed before being read
        return int(f'{self._state:0{len(self)}b}'[::-1], 2)
    @state.setter
    def state(self, state):
        # ensure seed is in the range [1, 2**m-1]
        #if (state < 1) or (state > len(self)):
            #state = 1 + state % (2**len(self)-2)
        self._state = int(f'{state:0{len(self)}b}'[::-1], 2)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        '''Execute a LFSR step and returns the output bit (bool)'''
        self._state = ((self._state << 1) | self._feedback) & self._smask
        self._output = self._state >> (self._length-1)
        self._feedback = self._parity(self._state & self._poly)
        return bool(self._output)
    
    def __len__(self):
        return self._length
    
    def run_steps(self, N=1):
        '''
        Run given number of steps and return the output stream
        '''
        return [next(self) for _ in range(n)]
    
    def cycle(self, state=None):
        if state is not None:
            self.state = state
        return self.run_steps(2**len(self) - 1)
    
    def __str__(self):
        return None

In [11]:
lfsr = LFSR([3, 1, 0], 4)

print("State |  b  |  fb")
print(f' {lfsr.state:0{len(lfsr)}b}  |  {lfsr.output}  |  {lfsr.feedback}')

for _ in islice(lfsr, 10):
    print(f' {lfsr.state:0{len(lfsr)}b}  |  {lfsr.output}  |  {lfsr.feedback}')

State |  b  |  fb
 100  |  0  |  1
 110  |  0  |  1
 111  |  1  |  0
 011  |  1  |  1
 101  |  1  |  0
 010  |  0  |  0
 001  |  1  |  1
 100  |  0  |  1
 110  |  0  |  1
 111  |  1  |  0
 011  |  1  |  1


In [2]:
poly = [5, 2, 0]
p = bin(sum([2**i for i in poly]))
print(p)

0b100101
